# RAG vectorstore

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pprados/langchain-rag/blob/master/docs/integrations/vectorstores/rag_vectorstore.ipynb)


RAG architectures are very popular. They can be quickly demonstrated on a few documents. For a production project, more effort is needed to achieve an effective architecture.

The basic principle is to divide a document into chunks, place them in a vector database, then select the chunks closest to the question and inject them into a prompt.

When splitting documents for retrieval, there are often conflicting desires:

1. You may want to keep documents small, ensuring that their embeddings accurately represent their meaning. If they become too long, the embeddings can lose their meaning.
2. You also want to maintain documents long enough to retain the context of each chunk.

When you have a lot of documents, and therefore a lot of pieces, it's likely that dozens of pieces have a distance close to the question. Taking only the top 4 is not a good idea. The answer may lie in the 6 or 7 tracks. How can we improve the match between the question and a fragment? By preparing several versions of the fragment, each with an embedding. In this way, one of the versions can be closer to the question than the original fragment. This version is stripped of context. But the context is still needed to answer the question correctly. One strategy consists of breaking down each fragment into different versions, but using the retriever to return to the original fragment. 

The `RAGVectorStore` strikes a balance by splitting and storing small chunks and different variations of data. During retrieval, it initially retrieves the small chunks but then looks up the parent IDs for those chunks and returns the larger documents.

The challenge lies in correctly managing the lifecycle of the three levels of documents:
- Original documents
- Chunks extracted from the original documents
- Transformations of chunks to generate more vectors for improved retrieval

The `RAGVectorStore`, in combination with other components, is designed to address this challenge.

For the sample, we are using the set of documents from Wikipedia.
We would like to answer questions related to mathematics.

> This notebook is not intended to demonstrate the relevance of each optimisation approach. The example is based on too few documents for that. It is simply intended to illustrate an implementation of an advanced RAG.

In [1]:
%pip install -q --upgrade pip
%pip install -q chromadb 'numpy<2.0.0' kaleido python-multipart wikipedia lark cohere
%pip install -q langchain_rag langchain_core langchain_community langchain_chroma langchain-openai langchain_qa_with_references

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
query = "What is the difference between pure and applied mathematics?"

# Prepare the environment

To start with, we create a working directory to store all sorts of things.

Then we add a small function to display lists of documents, with a selection of metadata.

In [3]:
from langchain_core.documents import Document
import logging
import pathlib
import tempfile
import tiktoken
from typing import List, Union

# Activate logging and prints
logging.basicConfig(level=logging.WARN)

CALLBACKS = []
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def pretty_print_docs(
        docs: Union[str, List[Document]], metadatas=[], kind: str = "Variations"
):
    def print_metadata(d):
        s = ",\n".join(
            [f"{metadata}={repr(d.metadata.get(metadata))}" for metadata in metadatas]
        )
        if s:
            return f"\n\033[92m{s}\033[0m"
        return ""

    def print_doc(d, i):
        r = f"\033[94m{kind} {i + 1}:\n{d.page_content[:80]}"
        if len(d.page_content) > 80:
            r += f"...[:{max(0, len(d.page_content) - 80)}]"
        r+=f" {len(encoding.encode(d.page_content))} toks"
        r += f"\033[0m{print_metadata(d)}"
        return r

    if isinstance(docs, list):
        print(f"\n{'-' * 40}\n".join([print_doc(d, i) for i, d in enumerate(docs)]))
    else:
        print(f"\033[92m{docs}\033[0m")

ROOT_PATH = tempfile._gettempdir() + "/rag"
pathlib.Path(ROOT_PATH).mkdir(exist_ok=True)
ROOT_PATH

'/tmp/rag'

# Select providers

In [4]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""  # Set api key"

## Select the embedding implementation

In [5]:
# Select OpenAI implementation
from langchain_openai import *

model_id="gpt-4o"

openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
# Add a cache for embeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

CACHE_EMBEDDING_PATH = ROOT_PATH + "/cache_embedding"
fs = LocalFileStore(CACHE_EMBEDDING_PATH)


embeddings = CacheBackedEmbeddings.from_bytes_store(
    openai_embeddings,
    fs,
    namespace=openai_embeddings.model if hasattr(openai_embeddings, "model") else "unknown",
)

In [7]:
# Calculates the parameters
nb_documents_to_import = 5  # How many documents should be imported from Wikipedia?
top_k = 4  # How many chunks should be injected in the prompt to answer the question?
doc_content_chars_max=4000   # First chars for wikipedia docs

embeddings_tokens_limit= openai_embeddings.embedding_ctx_length

context_size = OpenAI.modelname_to_contextsize(model_id)  # The GPT3.5 limit

# 10% for the prompt without context
prompt_tokens = int(context_size * (10 / 100))  

# 20% for the response
output_tokens = min(16384,int(context_size * (20 / 100)))

# Minimum tokens for one document
min_doc_tokens = 200

# Maximum size for each documents to inject
doc_tokens = (context_size - prompt_tokens - output_tokens ) // top_k
if doc_tokens > embeddings_tokens_limit:
    top_k = (context_size - prompt_tokens - output_tokens ) // embeddings_tokens_limit
elif doc_tokens < min_doc_tokens:
    top_k = (context_size - prompt_tokens - output_tokens ) // min_doc_tokens

# Then, the maximum nomber of tokens for the prompt
input_tokens = context_size - output_tokens

print(f"{top_k=} {context_size=} {prompt_tokens=}, {doc_tokens=}, {input_tokens=}, {output_tokens=}")

top_k=12 context_size=128000 prompt_tokens=12800, doc_tokens=24704, input_tokens=111616, output_tokens=16384


## Select the LLM
Before starting, we need to:
- Set the environment variables
- Choose a language model (LLM), determine the context size, and set the maximum number of tokens for generation
- Enable all caches

In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model=model_id,
    temperature=0.2,
    max_tokens=output_tokens,  # Maximum possible
)

In [9]:
# Add a cache
from langchain_core import globals
from langchain_community.cache import SQLiteCache

LANCHAIN_CACHE_PATH = ROOT_PATH + "/cache_llm"
globals.set_llm_cache(SQLiteCache(database_path=LANCHAIN_CACHE_PATH))

# Load the documents

We want to retrieve documents about mathematics from wikipedia.

In [10]:
from langchain_community.retrievers import WikipediaRetriever

documents = WikipediaRetriever(
    top_k_results=nb_documents_to_import, 
    doc_content_chars_max=doc_content_chars_max
).invoke("mathematic")
pretty_print_docs(documents, ["title"],kind="Documents")

/home/pprados/workspace.bda/langchain-rag/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/pprados/workspace.bda/langchain-rag/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Documents 1:
Mathematics is a field of study that discovers and organizes methods, theories a...[:3920] 714 toks
title='Mathematics'
----------------------------------------
Documents 2:
The history of mathematics deals with the origin of discoveries in mathematics a...[:3920] 800 toks
title='History of mathematics'
----------------------------------------
Documents 3:
Subtraction (which is signified by the minus sign, –) is one of the four arithme...[:3920] 889 toks
title='Subtraction'
----------------------------------------
Documents 4:
Doing Hard Time is a 2004 drama crime film starring Boris Kodjoe, Michael K. Wil...[:2336] 579 toks
title='Doing Hard Time'


# Transform documents
The idea is to transform a document into multiple versions and calculate a vector for each one.

In [11]:
from langchain_text_splitters import *
from langchain_rag.document_transformers import *
from langchain_rag.document_transformers import DocumentTransformerPipeline

The first step is to split the document to ensure compatibility with the `max_input_tokens`. This could be a transformation pipeline, for an initial wiki split, followed by a size split for example.

In [12]:
wiki_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "={1,6} .* ={1,6}",  # See https://en.wikipedia.org/wiki/Help:Wikitext
        "\n----+\n",            
        "\n\n",
        "\n",
    ],
    chunk_size=doc_content_chars_max,
    chunk_overlap=0,
    is_separator_regex=True)

token_splitter = TokenTextSplitter(chunk_size=doc_tokens, chunk_overlap=0)
parent_transformer = DocumentTransformerPipeline(transformers=[wiki_splitter,token_splitter])

Let's test the transformations.

In [13]:
chunk_documents = parent_transformer.transform_documents(documents)
f"before:{len(documents)} documents, after:{len(chunk_documents)} chunks"

'before:4 documents, after:4 chunks'

In [14]:
pretty_print_docs(chunk_documents, kind="Chunk")

Chunk 1:
Mathematics is a field of study that discovers and organizes methods, theories a...[:3920] 714 toks
----------------------------------------
Chunk 2:
The history of mathematics deals with the origin of discoveries in mathematics a...[:3920] 800 toks
----------------------------------------
Chunk 3:
Subtraction (which is signified by the minus sign, –) is one of the four arithme...[:3920] 889 toks
----------------------------------------
Chunk 4:
Doing Hard Time is a 2004 drama crime film starring Boris Kodjoe, Michael K. Wil...[:2336] 579 toks


We need multiple variations for each chunk.

In [15]:
chunk_transformer = DocumentTransformers(
    transformers=[
        GenerateQuestionsTransformer.from_llm(llm),
        SummarizeTransformer.from_llm(llm),
        CopyDocumentTransformer(),
    ]
)

> **Note:** that we require all transformations for each chunk, including the original chunk. This is why we include the `CopyDocumentTransformer()`.

Now, let's test the transformation for the first document.

In [16]:
variations_of_chunks = chunk_transformer.transform_documents(chunk_documents[:1])
# Select the variations for the first chunk
pretty_print_docs(variations_of_chunks)

Variations 1:
What are the main areas of mathematics mentioned in the text? 12 toks
----------------------------------------
Variations 2:
How did the foundational crisis in mathematics at the end of the 19th century im...[:47] 24 toks
----------------------------------------
Variations 3:
What role does mathematics play in fields such as natural sciences, engineering,...[:13] 17 toks
----------------------------------------
Variations 4:
SUMMARY:
Mathematics is a discipline that develops and organizes methods, theori...[:1099] 213 toks
----------------------------------------
Variations 5:
Mathematics is a field of study that discovers and organizes methods, theories a...[:3920] 714 toks


We see 3 questions, a summary of the chunk and the original chunk.

![Tree of variations](plantuml/variations.png)

# Saving all Variations in a Vector Store
Now, our goal is to store the chunks and their respective variations in a vector store. During retrieval, the process begins by fetching the smaller chunks but then involves looking up the parent IDs for those chunks and returning the original chunk.

A specialized vector store is designed for this purpose: the `RAGVectorStore`.
It's not a standalone vector store but rather a wrapper for another vector store. When you add a document, the document undergoes transformation with the `parent_transformer`, and each chunk is enriched with various versions through the `chunk_transformer`. Each parameter is optional.

## Build step by step
First, we need to create some persistent components:
- A standard vector store
- A `Docstore` to store each original chunk returned by the *retriever* and the relationship between the document and chunks.

In [17]:
from langchain_chroma.vectorstores import Chroma
import chromadb.segment.impl.metadata.sqlite
import chromadb.segment.impl.vector.local_persistent_hnsw
chromadb.segment.impl.metadata.sqlite.logger.setLevel(logging.ERROR)
chromadb.segment.impl.vector.local_persistent_hnsw.logger.setLevel(logging.ERROR)

VS_PATH = ROOT_PATH + "/vs"
chroma_vectorstore = Chroma(
    collection_name="all_variations_of_chunks",
    embedding_function=embeddings,
    persist_directory=VS_PATH,
)

In [18]:
DOCSTORE_PATH = ROOT_PATH + "/chunks"
from langchain.storage import EncoderBackedStore,LocalFileStore
import pickle

docstore = EncoderBackedStore[str, Document](
    store=LocalFileStore(root_path=DOCSTORE_PATH),
    key_encoder=lambda x: x,
    value_serializer=pickle.dumps,
    value_deserializer=pickle.loads,
)

All documents must have a unique ID in their metadata. 
Then, it's possible to use the advanced `RAGVectorStore`. 
It's a wrapper around a standard vector store, specialized for managing different transformations and the lifecycle of documents.

In [19]:
from langchain_rag.vectorstores import RAGVectorStore

variation_k = 10
rag_vectorstore = RAGVectorStore(
    vectorstore=chroma_vectorstore,
    docstore=docstore,
    source_id_key="source",  # Uniq id of documents
    # parent_transformer=parent_transformer, # FIXME
    # chunk_transformer=chunk_transformer,
    #search_kwargs={"k": variation_k},
)

Now, it's time to add documents to this *special* vector store.
- If the `parent_transformer` is set, the document is transformed into a new list of chunk documents (generally, this is a split phase).
- Then, if the `chunk_transformer` is set, each chunk document is transformed to generate some variations.
- Each transformation of all chunks is added to the destination vector store (in this case, it's referred to as "chroma").
- All chunks are saved in the `Docstore` with the list of all associated variations.
- All IDs of chunks generated for each document are saved in the `Docstore`. This makes it possible to remove the document and all associated chunks when needed.
- `variation_k` variations is returned in the delegate vectorstore

> This takes time, because the transformations are carried out when the documents are added. Among other things, an LLM must be invoked to produce a summary of each chunk.

In [20]:
ids = rag_vectorstore.add_documents(documents)
ids

['1843cc78-8208-4c7e-b99e-e7525d54469d',
 '96116cf0-3d90-4f71-ac9e-369c208e32eb',
 '6aaba945-de53-43e6-a64b-f6a415b86da2',
 'd576dc10-5453-4a2b-b625-f50381faefff']

While conducting the search, an embedding is computed for the query and subsequently compared to the embeddings of all the transformed chunks. The metadata for each transformed chunk contains a reference to the ID of the original chunk, allowing for the retrieval of the respective chunk.

The IDs returned by `add_documents()` consist of a list of `document IDs`. You can utilize these IDs to remove all documents, related chunks and variations.

When you examine the langchain API, you may wonder where to store the document IDs from the vector store.

In [21]:
pretty_print_docs(
    rag_vectorstore.search(query=query, search_type="similarity"),
    ["source", "_chunk_id"],
    kind="Chunk",
)

Chunk 1:
Mathematics is a field of study that discovers and organizes methods, theories a...[:3920] 714 toks
source='https://en.wikipedia.org/wiki/Mathematics',
_chunk_id=None
----------------------------------------
Chunk 2:
Mathematics is a field of study that discovers and organizes methods, theories a...[:432] 97 toks
source='https://en.wikipedia.org/wiki/Mathematics',
_chunk_id='5d91e8e8-fdcf-4786-b756-46b6ddfc89a5'
----------------------------------------
Chunk 3:
 involves the description and manipulation of abstract objects that consist of e...[:408] 87 toks
source='https://en.wikipedia.org/wiki/Mathematics',
_chunk_id='36566cca-bae3-4e7a-b1d4-6a162d209b43'
----------------------------------------
Chunk 4:
The history of mathematics deals with the origin of discoveries in mathematics a...[:3920] 800 toks
source='https://en.wikipedia.org/wiki/History_of_mathematics',
_chunk_id=None


In [22]:
# Delete documents, chunks and variations
rag_vectorstore.delete(ids=ids)

# Index Vector Store
To manage the lifecycle of the documents in the vector store, you can utilize an `index()`.
A `RecordManager` can keep track of the evolution of each document. Use langchain `index()` to import the documents.

In [23]:
from langchain.indexes import index, SQLRecordManager

record_manager = SQLRecordManager(
    namespace="record_manager_cache", db_url=f"sqlite:///{ROOT_PATH}/record_manager.db"
)
record_manager.create_schema()

In [24]:
# Save all the information in:
# - record manager
# - docstore
# - vectorstore
index_kwargs = {
    "record_manager": record_manager,
    "vector_store": rag_vectorstore,
    "source_id_key": "source",
}
result = index(docs_source=documents, cleanup="incremental", **index_kwargs)
result

{'num_added': 4, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

## Alternative factory
To simplify the creation of the persistance ecosystem, you can use the `from_vs_in_memory` method for in-memory usage only, and `from_vs_in_sql` for usage with SQL.

```python
rag_vectorstore, index_kwargs = RAGVectorStore.from_vs_in_memory(
    vectorstore=chroma_vectorstore,
    parent_transformer=parent_transformer,
    chunk_transformer=chunk_transformer,
    source_id_key="source",
)
index(
    docs_source=documents,
    cleanup="incremental",
    **index_kwargs
)
```

```python
rag_vectorstore, index_kwargs = RAGVectorStore.from_vs_in_sql(
    vectorstore=chroma_vectorstore,
    parent_transformer=parent_transformer,
    chunk_transformer=chunk_transformer,
    source_id_key="source",
    db_url=f"sqlite:///{ROOT_PATH}/record_manager.db",
)
index(
    docs_source=documents,
    cleanup="incremental",
    **index_kwargs
)
```

If you import the same documents, you will notice that all documents are skipped. 
> Without using `index()`, in a classical vector store, the same document will be present twice. This has the same effect as dividing the `top_k` by two during the search! Because 2 different documents with the same content are returned by the vectorstore.

In [25]:
result = index(docs_source=documents, cleanup="incremental", **index_kwargs)
result

{'num_added': 0, 'num_updated': 0, 'num_skipped': 4, 'num_deleted': 0}

If your document is changed, the previous version is deleted.
> **Note:** Only the updated document is transformed ! So you save on treatments.

In [26]:
documents[0].page_content += " Is changed."
result = index(docs_source=documents, cleanup="incremental", **index_kwargs)
result

{'num_added': 1, 'num_updated': 0, 'num_skipped': 3, 'num_deleted': 1}

To delete the old records, use the `full` strategy.

In [27]:
del documents[-1]
result = index(docs_source=documents, cleanup="full", **index_kwargs)
result

{'num_added': 0, 'num_updated': 0, 'num_skipped': 3, 'num_deleted': 1}

To delete the all records, use the full strategy and an empty list.

In [28]:
result = index(docs_source=[], cleanup="full", **index_kwargs)
result

{'num_added': 0, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 3}

In [29]:
# Re-import all documents
result = index(docs_source=documents, cleanup="incremental", **index_kwargs)
result

{'num_added': 3, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

It's important to note that there are three ways to save parts of the data:

- In the *vector store*: this includes the bucket, metadata, and the associated embedding vectors.
- In the *doc store*: this covers the original bucket and the relationship between parent and chunks before the *chunk transformations*.
- In the *SQLRecordManager*: this involves the references of the parent document or chunks.

> **Note:** Each source does not manage transactions. If a problem occurs while adding a document, it is highly likely that the sources will be inconsistent.

# Use advanced retrievers
Just like with the standard vector store, you can convert the `RAGVectorStore` into a `Retriever`.

In [30]:
rag_retriever = rag_vectorstore.as_retriever()
selected_chunks = rag_retriever.get_relevant_documents(query)
pretty_print_docs(selected_chunks, ["source", "_chunk_id", "title"], "Chunk")

/tmp/ipykernel_91210/3039810368.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  selected_chunks = rag_retriever.get_relevant_documents(query)


Chunk 1:
Mathematics is a field of study that discovers and organizes methods, theories a...[:3932] 717 toks
source='https://en.wikipedia.org/wiki/Mathematics',
_chunk_id=None,
title='Mathematics'
----------------------------------------
Chunk 2:
Mathematics is a field of study that discovers and organizes methods, theories a...[:432] 97 toks
source='https://en.wikipedia.org/wiki/Mathematics',
_chunk_id='5d91e8e8-fdcf-4786-b756-46b6ddfc89a5',
title='Mathematics'
----------------------------------------
Chunk 3:
 involves the description and manipulation of abstract objects that consist of e...[:408] 87 toks
source='https://en.wikipedia.org/wiki/Mathematics',
_chunk_id='36566cca-bae3-4e7a-b1d4-6a162d209b43',
title='Mathematics'
----------------------------------------
Chunk 4:
The history of mathematics deals with the origin of discoveries in mathematics a...[:3920] 800 toks
source='https://en.wikipedia.org/wiki/History_of_mathematics',
_chunk_id=None,
title='History of mathematics'


# Specialized Retrievers
It's possible to combine multiple retrievers or use specialized retrievers for advanced applications.

## SelfQueryRetriever
The `SelfQueryRetriever` can generate a metadata filter. We use it to provide the option to filter the chunks by the title of the original document.

In [31]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="title",
        description="The title of the document.",
        type="string",
    ),
]
document_content_description = "Documents on mathematics"
self_retriever = SelfQueryRetriever.from_llm(
    llm,
    rag_vectorstore,
    document_content_description,
    metadata_field_info,
    use_original_query=True,
    verbose=True,
)

pretty_print_docs(
    self_retriever.get_relevant_documents(
        "In the document 'History of mathematics', " + query
    ),
    ["title"],
    kind="Chunk",
)

Chunk 1:
The history of mathematics deals with the origin of discoveries in mathematics a...[:429] 99 toks
title='History of mathematics'
----------------------------------------
Chunk 2:
The history of mathematics deals with the origin of discoveries in mathematics a...[:3920] 800 toks
title='History of mathematics'
----------------------------------------
Chunk 3:
 seems to be the most ancient and widespread mathematical development after basi...[:4] 13 toks
title='History of mathematics'
----------------------------------------
Chunk 4:
 also in the field of astronomy to record time and formulate calendars.
The earl...[:327] 104 toks
title='History of mathematics'


> **Note:** We set `use_original_query` because, otherwise, the question can be modified if there is no filter on the metadata to be applied. Which is irrelevant (see [ticket](https://github.com/langchain-ai/langchain/pull/9309) )

## MergerRetriever
With filter, we can obtain a retriever specialized in summaries. But then you have to use chroma directly, in order to query the variations.

In [32]:
summary_retriever = chroma_vectorstore.as_retriever(
    search_kwargs={"filter": {"transformer": {"$eq": 'SummarizeTransformer'}}}
)
pretty_print_docs(summary_retriever.get_relevant_documents(query), ["transformer"])

Just for the demo, we will combine it with the chunk retriever.

In [33]:
from langchain.retrievers.merger_retriever import MergerRetriever

merge_retriever = MergerRetriever(retrievers=[self_retriever, summary_retriever])
pretty_print_docs(
    merge_retriever.get_relevant_documents(query), ["transformer"], kind="Chunk"
)

Chunk 1:
Mathematics is a field of study that discovers and organizes methods, theories a...[:3932] 717 toks
transformer=None
----------------------------------------
Chunk 2:
Mathematics is a field of study that discovers and organizes methods, theories a...[:432] 97 toks
transformer=None
----------------------------------------
Chunk 3:
 involves the description and manipulation of abstract objects that consist of e...[:408] 87 toks
transformer=None
----------------------------------------
Chunk 4:
The history of mathematics deals with the origin of discoveries in mathematics a...[:3920] 800 toks
transformer=None


## MultiQueryRetriever
Retrieval results may vary with minor changes in query phrasing or if the embeddings do not accurately capture the data's semantics. The `MultiQueryRetriever` streamlines the prompt-tuning process by employing an LLM to generate multiple queries from diverse perspectives based on a user input query. For each query, it retrieves a collection of pertinent documents and combines the unique results from all queries to obtain a larger set of potentially relevant documents.

In [34]:
from langchain.retrievers.multi_query import MultiQueryRetriever,logger
logger.setLevel(logging.INFO)

query = "What is the difference between pure and applied mathematics?"

# Generate 3 questions from the user questions, and these version to find a better candidats in vectorstore
multi_query_retriever = MultiQueryRetriever.from_llm(
    llm=llm,
    retriever=merge_retriever,
)

pretty_print_docs(multi_query_retriever.get_relevant_documents(query), ["transformer"],kind="Chunk")
final_retriever = multi_query_retriever

INFO:langchain.retrievers.multi_query:Generated queries: ['How do pure mathematics and applied mathematics differ from each other?  ', 'What are the main distinctions between applied mathematics and pure mathematics?  ', 'In what ways do the goals and methods of pure mathematics contrast with those of applied mathematics?']


Chunk 1:
Mathematics is a field of study that discovers and organizes methods, theories a...[:3932] 717 toks
transformer=None
----------------------------------------
Chunk 2:
Mathematics is a field of study that discovers and organizes methods, theories a...[:432] 97 toks
transformer=None
----------------------------------------
Chunk 3:
 involves the description and manipulation of abstract objects that consist of e...[:408] 87 toks
transformer=None
----------------------------------------
Chunk 4:
The history of mathematics deals with the origin of discoveries in mathematics a...[:3920] 800 toks
transformer=None
----------------------------------------
Chunk 5:
The history of mathematics deals with the origin of discoveries in mathematics a...[:429] 99 toks
transformer=None


## Others...
Other strategies can be added. This depends on the project, and above all on their relevance to the quality of the documents handled.

- The `EnsembleRetriever` takes a list of retrievers as input and ensemble the results of their `get_relevant_documents()` methods and rerank the results based on the [Reciprocal Rank Fusion algorithm](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).
- ...

At this stage, when we employ the retriever:

- Multiple queries are generated to locate the relevant documents (via `multi_query_retriever`).
- For each query:
    - Variations are used for better selection of chunks (via ̀`RagVectorstore`)
    - Both the original chunk and the chunk summary are retrieved (via `MergeRetriever`).
    - If feasible, a metadata filter is applied (via `SelfQueryRetriever`)
- Only this selected candidate can be used to answer a question.

# Use a compressor
It's possible to use a *compressor*, to filter the selection, and reduce the volume of each chunks.

You can combine some filters in a pipeline.
- The [EmbeddingsFilter](https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression#embeddingsfilter) can add a similarity threshold between the query and documents
- The [CohereRerank](https://python.langchain.com/docs/integrations/retrievers/cohere-reranker) can re-rank the chunks (Need API key).
- The [LLMChainFilter](https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression#llmchainfilter) decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.
- THe [LongContextReorder](https://python.langchain.com/docs/integrations/retrievers/merger_retriever#re-order-results-to-avoid-performance-degradation) to reorder the selected documents
- ...

In [35]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings,
    similarity_threshold=0.7,  # Threshold for determining when two documents are redundant.
)

In [36]:
from langchain_community.document_transformers import LongContextReorder

long_context_reorder = LongContextReorder()

In [37]:
# Combine compressors
from langchain.retrievers.document_compressors import DocumentCompressorPipeline

compressor = DocumentCompressorPipeline(
    transformers=[
        # embeddings_filter, # Deactivated, so as not to conflict with RAGVectorstore
        long_context_reorder,
    ]
)

> **Note:** We do not use `embeddings_filter`, because a fragment can have a proximity < 0.7, but one of its variations a higher proximity. We want to keep the fragment in this case.

Now, we can add a filter with our pipeline.

In [38]:
from langchain.retrievers import ContextualCompressionRetriever

query = "What is the difference between pure and applied mathematics?"

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=multi_query_retriever
)

pretty_print_docs(compression_retriever.get_relevant_documents(query))

INFO:langchain.retrievers.multi_query:Generated queries: ['How do pure mathematics and applied mathematics differ from each other?  ', 'What are the main distinctions between applied mathematics and pure mathematics?  ', 'In what ways do the goals and methods of pure mathematics contrast with those of applied mathematics?']


Variations 1:
Mathematics is a field of study that discovers and organizes methods, theories a...[:3932] 717 toks
----------------------------------------
Variations 2:
 involves the description and manipulation of abstract objects that consist of e...[:408] 87 toks
----------------------------------------
Variations 3:
The history of mathematics deals with the origin of discoveries in mathematics a...[:429] 99 toks
----------------------------------------
Variations 4:
The history of mathematics deals with the origin of discoveries in mathematics a...[:3920] 800 toks
----------------------------------------
Variations 5:
Mathematics is a field of study that discovers and organizes methods, theories a...[:432] 97 toks


In [39]:
final_retriever = compression_retriever

The final map is:

![Chain of retrievers](plantuml/all_retrievers.png)

# Asking a Question

Now, it's possible to utilize this architecture to pose a question. We hope that all these optimisations will lead to a better selection of chunks, based on the user's question.

A problem can arise if the number of documents to be analysed is too large for the size of the prompt.
Several strategies are available to manage this, identified by the 
[`chain_type`](https://python.langchain.com/docs/use_cases/question_answering/vector_db_qa#chain-type) parameter.

> **Note 1**: The version `load_qa_chain()` and `RetrievalQAWithSourcesChain` are subject to hallucinations. They can respond without using the documents provided. This is not the case for `RetrievalQAWithReferencesChain` and `RetrievalQAWithReferencesAndVerbatimsChain`.

> **Note 2**: The `map_reduce` chain type, use an approach similar to *compressor*, but working recursively to keep the number of tokens below a threshold. 

In [40]:
from langchain.chains.question_answering import load_qa_chain
query = "What is the difference between pure and applied mathematics?"

chain = load_qa_chain(
    llm,
    
    chain_type="map_reduce",  # "stuff", "map_reduce", "refine", "map_rerank"
)
result = chain.invoke(
    {
        "input_documents": final_retriever.get_relevant_documents(query),
        "question": query,
    },
    callbacks=CALLBACKS,
)
print(result["output_text"])

/tmp/ipykernel_91210/3698088474.py:4: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(
INFO:langchain.retrievers.multi_query:Generated queries: ['How do pure mathematics and applied mathematics differ from each other?  ', 'What are the main distinctions between applied mathematics and pure mathematics?  ', 'In what ways do the goals and methods of pure mathematics contrast with those of applied mathematics?']


Pure mathematics is developed independently from any application and focuses on abstract concepts and theoretical frameworks. Applied mathematics, on the other hand, is developed in close correlation with practical applications and is often used to model real-world phenomena.


If the documents have `sources` and the URLs are not too large, you can use `RetrievalQAWithSourcesChain`.

In [41]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="map_reduce",  # "stuff", "map_reduce", "refine", "map_rerank"
    retriever=final_retriever,
    callbacks=CALLBACKS,
)
result = chain.invoke(query)
print(result["answer"])
pretty_print_docs(result["sources"])

INFO:langchain.retrievers.multi_query:Generated queries: ['How do pure mathematics and applied mathematics differ from each other?  ', 'What are the main distinctions between applied mathematics and pure mathematics?  ', 'In what ways do the goals and methods of pure mathematics contrast with those of applied mathematics?']


Pure mathematics is developed independently from any application, focusing on abstract concepts and theoretical frameworks. In contrast, applied mathematics is developed in close correlation with practical applications, such as statistics and game theory, which are used to solve real-world problems.


https://en.wikipedia.org/wiki/Mathematics


In [42]:
# Clean up
import shutil

shutil.rmtree(ROOT_PATH)

# References
- [Why Your RAG Is Not Reliable in a Production Environment](https://towardsdatascience.com/why-your-rag-is-not-reliable-in-a-production-environment-9e6a73b3eddb)
- [Forget RAG, the Future is RAG-Fusion](https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1)
- [A first intro to Complex RAG](https://medium.com/enterprise-rag/a-first-intro-to-complex-rag-retrieval-augmented-generation-a8624d70090f)
- [Advanced RAG Techniques: an Illustrated Overview](https://pub.towardsai.net/advanced-rag-techniques-an-illustrated-overview-04d193d8fec6)
- [weblangchain](https://blog.langchain.dev/weblangchain/)
- [How Chunk Sizes Affect Semantic Retrieval Results](https://ai.plainenglish.io/investigating-chunk-size-on-semantic-results-b465867d8ca1)